In [21]:
import pandas as pd

In [23]:
def read_files_into_df(input_filename, output_filename):
    df_input = pd.read_csv(input_filename, names=['packet_in', 'time_in', 'initial'])
    df_input['message'] = df_input['packet_in'].apply(lambda x : int(x[:48], 2))

    df_output = pd.read_csv(output_filename, names=['packet_out', 'time_out', 'final'])
    df_output['message'] = df_output['packet_out'].apply(lambda x : int(x[:48], 2))

    joined_df = df_input.join(df_output, on='message', lsuffix='left')
    joined_df = joined_df.drop(columns=['messageleft', 'packet_in', 'packet_out'])

    joined_df['time_in'] = joined_df['time_in'].apply( lambda x : int(x.split(' ')[0]) )
    joined_df['time_out'] = joined_df['time_out'].apply( lambda x : int(x.split(' ')[0]) )
    joined_df['difference'] = joined_df['time_out']-joined_df['time_in']

    return joined_df

data = read_files_into_df('mesh_input_1.txt', 'mesh_output_1.txt')
data

,time_in,initial,time_out,final,message,difference
0,5000,(0.0),80000,(0.1),0,75000
1,11000,(0.0),156000,(0.1),1,145000
2,49000,(0.0),232000,(0.1),2,183000
3,125000,(0.0),308000,(0.1),3,183000
4,201000,(0.0),384000,(0.1),4,183000
...,...,...,...,...,...,...
1315,198354000,(3.3),198537000,(3.2),1315,183000
1316,198430000,(3.3),198613000,(3.2),1316,183000
1317,198506000,(3.3),198689000,(3.2),1317,183000
1318,198582000,(3.3),198765000,(3.2),1318,183000


In [18]:
def get_detailed_info(initial_coords: tuple, dataframe: pd.DataFrame):
    initial_str = f"({initial_coords[0]}.{initial_coords[1]})"

    df = dataframe[dataframe["initial"] == initial_str]

    # first message travelling in the system
    table = df[df["message"] % 10 == 0][["initial", "final", "difference"]]
    table = table.rename(columns={"difference": "first-diff"})

    # second message travelling in the system
    sec = df[df["message"] % 10 == 1][["initial", "final", "difference"]]
    table = table.merge(sec, on=["final", "initial"]).rename(columns={"difference": "second-diff"})


    # last message travelling in the system
    last = df[df["message"] % 10 == 9][["initial", "final", "difference"]]
    table = table.merge(last, on=["final", "initial"]).rename(columns={"difference": "last-diff"})

    # to find the number of messages before it stabilizes
    max_value = df.groupby("final")["difference"].max()
    # basically the same as `table['last-diff']` ^
    df_and_max_value = pd.merge(df, max_value, on='final')
    df_only_not_max_value = df_and_max_value[df_and_max_value["difference_y"] != df_and_max_value["difference_x"]]
    count = df_only_not_max_value.groupby('final').count()["difference_y"]
    table = pd.merge(count, table, on=["final"]).rename(columns={"difference_y": "msgs-until-full"})
    #table = table.unique()

    table.drop_duplicates(subset=["final"], keep=False,inplace=True)

    return table

info_00 = get_detailed_info((0,0), data)
info_00


,final,msgs-until-full,initial,first-diff,second-diff,last-diff
0,(0.1),2,(0.0),75000,145000,183000
1,(0.2),7,(0.0),119000,189000,308000
2,(0.3),2,(0.0),138000,191000,212000
3,(1.0),2,(0.0),75000,145000,183000
4,(1.3),2,(0.0),119000,185000,219000
5,(2.0),7,(0.0),119000,189000,308000
6,(2.3),3,(0.0),118000,184000,252000
7,(3.0),2,(0.0),138000,191000,212000
8,(3.1),2,(0.0),119000,185000,219000
9,(3.2),3,(0.0),118000,184000,252000


In [19]:
# info01 = get_detailed_info((0,1), data) 
# info01
# weird data ^

info10 = get_detailed_info((1,0), data)
info10

,final,msgs-until-full,initial,first-diff,second-diff,last-diff
0,(0.0),1,(1.0),75000,137000,137000
1,(0.1),2,(1.0),68000,121000,129000
2,(0.2),7,(1.0),125000,195000,301000
3,(0.3),2,(1.0),144000,197000,205000
4,(1.3),7,(1.0),82000,122000,143000
5,(2.0),9,(1.0),100000,170000,234000
6,(2.3),2,(1.0),125000,191000,212000
7,(3.0),1,(1.0),119000,181000,181000
8,(3.1),2,(1.0),106000,172000,193000
9,(3.2),5,(1.0),86000,139000,168000


In [20]:
get_detailed_info((3,0), data)


,final,msgs-until-full,initial,first-diff,second-diff,last-diff
0,(0.0),2,(3.0),138000,191000,212000
1,(0.1),2,(3.0),119000,185000,219000
2,(0.2),3,(3.0),118000,184000,252000
3,(0.3),3,(3.0),73000,122000,156000
4,(1.0),7,(3.0),119000,189000,308000
5,(1.3),3,(3.0),118000,184000,252000
6,(2.0),2,(3.0),75000,145000,183000
7,(2.3),2,(3.0),119000,185000,219000
8,(3.1),2,(3.0),75000,145000,183000
9,(3.2),7,(3.0),119000,189000,308000
